# Исследование корреляции солнечной активности и биржи

In [1]:
import pandas as pd

In [2]:
#url = 'https://www.spaceweather.gc.ca/solar_flux_data/daily_flux_values/fluxtable.txt'
df = pd.read_csv('fluxtable.txt', delim_whitespace=True, skiprows=[1])

# Объединяем fluxdate и fluxtime в одно поле timestamp
df['timestamp'] = pd.to_datetime(df['fluxdate'].astype(str) + df['fluxtime'].astype(str), format='%Y%m%d%H%M%S')

# Удаляем старые колонки fluxdate и fluxtime
df.drop(columns=['fluxdate', 'fluxtime'], inplace=True)

# Перемещаем колонку timestamp в начало
df = df[['timestamp'] + [col for col in df.columns if col != 'timestamp']]
df.head(3)

,timestamp,fluxjulian,fluxcarrington,fluxobsflux,fluxadjflux,fluxursi
0,2004-10-28 17:00:00,2453307.229,2022.605,132.7,130.9,117.8
1,2004-10-28 20:00:00,2453307.354,2022.610,135.8,134.0,120.6
2,2004-10-29 20:00:00,2453308.354,2022.646,130.6,128.8,115.9


In [3]:
solar=df.set_index('timestamp')[['fluxadjflux']].resample('D').mean().reset_index()
solar.head(3)

,timestamp,fluxadjflux
0,2004-10-28,132.450000
1,2004-10-29,130.550000
2,2004-10-30,139.466667


In [4]:
import yfinance as yf

In [5]:
tickers = '''AAPL, MSFT, AMZN, GOOGL, META, TSLA, JNJ, JPM, V, PG, NVDA, WMT, 
             DIS, BAC, MA, INTC, PFE, XOM, NFLX, ADBE, CRM, KO, PEP, GE'''

# Загружаем данные с конкретной начальной даты
raw_data = yf.download(
    tickers=tickers,
    start="2004-10-28",
    interval="1d",
    progress=False,
    group_by='ticker',
    threads=True  # Параллельная загрузка для ускорения
)

YF.download() has changed argument auto_adjust default to True


In [6]:
raw_data.head(3)

Ticker             KO                                                   WMT   
Price            Open       High        Low      Close    Volume       Open   
Date                                                                          
2004-10-28  10.942132  11.056084  10.849885  10.985542  12326400  11.877177  \
2004-10-29  11.096785  11.096785  10.988258  11.031669  16822600  11.901500   
2004-11-01  11.042526  11.064231  10.936713  10.996402  11649200  11.923615   

Ticker                                                 ...     GOOGL   
Price            High        Low      Close    Volume  ...      Open   
Date                                                   ...             
2004-10-28  12.016492  11.874965  11.939095  25229100  ...  4.654798  \
2004-10-29  12.009856  11.852850  11.923614  30020400  ...  4.959250   
2004-11-01  11.923615  11.828527  11.908136  26923800  ...  4.826099   

Ticker                                                     XOM              
Price           High       Low     Close     Volume       Open       High   
Date                                                                        
2004-10-28  4.847044  4.627868  4.819864  593278128  24.666629  24.920401  \
2004-10-29  4.985681  4.752542  4.753539  845653500  24.524514  24.996530   
2004-11-01  4.928830  4.769248  4.887937  488507004  25.072662  25.123416   

Ticker                                      
Price             Low      Close    Volume  
Date                                        
2004-10-28  24.488989  24.671705  18310900  
2004-10-29  24.524514  24.981304  14362400  
2004-11-01  24.620947  24.778286  13740100  

[3 rows x 120 columns]

In [7]:
# Извлекаем цены закрытия и преобразуем структуру
close_prices = (
    raw_data
    .xs('Close', level=1, axis=1)      # Выбираем уровень Close
    .sort_index()                       # Сортируем по времени
)
close_prices.head(3)

Ticker,KO,WMT,PFE,NVDA,DIS,NFLX,CRM,INTC,PG,AMZN,...,JPM,ADBE,JNJ,MSFT,AAPL,META,V,MA,GOOGL,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2004-10-28,10.985542,11.939095,11.836786,0.112572,19.968206,1.370000,5.009520,12.849730,29.297018,1.7225,...,22.336187,27.733749,32.668514,17.440018,0.785292,NaN,NaN,NaN,4.819864,24.671705
2004-10-29,11.031669,11.923614,11.935732,0.110585,20.047703,1.352857,5.051784,12.843958,29.222790,1.7065,...,22.405848,28.003637,32.876884,17.415112,0.788452,NaN,NaN,NaN,4.753539,24.981304
2004-11-01,10.996402,11.908136,11.873887,0.112955,20.055647,1.347143,5.394868,12.947818,29.120020,1.7550,...,22.347811,28.158575,32.910667,17.483599,0.789204,NaN,NaN,NaN,4.887937,24.778286


In [8]:
# Для солнечных данных
solar = df.set_index('timestamp')[['fluxadjflux']].resample('D').mean()

# Для цен акций (ваш текущий код)
close_prices = raw_data.xs('Close', level=1, axis=1).sort_index()

# Объединяем данные в один DataFrame
combined = solar.join(close_prices, how='inner').dropna()


combined.head(5)

,fluxadjflux,KO,WMT,PFE,NVDA,DIS,NFLX,CRM,INTC,PG,...,JPM,ADBE,JNJ,MSFT,AAPL,META,V,MA,GOOGL,XOM
2012-05-18,134.033333,24.911930,16.045567,13.038118,0.276960,38.746349,9.994286,36.192844,18.292822,44.061726,...,23.570284,31.309999,44.162266,23.397007,15.961046,38.084522,25.710611,36.651821,14.970756,48.813316
2012-05-21,128.333333,25.066687,16.202347,13.020785,0.281774,39.259312,10.248571,37.055531,18.348951,43.971550,...,22.880552,32.009998,44.245926,23.780693,16.890928,33.900501,26.527765,38.139969,15.312609,49.154808
2012-05-22,123.900000,24.959028,16.379690,12.922584,0.278335,39.259312,9.672857,37.154976,18.264751,43.805073,...,23.936258,32.009998,44.280785,23.788689,16.761232,30.882034,27.023083,38.405643,14.980729,49.100903
2012-05-23,119.966667,25.080141,16.598148,12.760834,0.285213,39.100113,10.272857,37.443363,17.850761,43.277874,...,24.112206,32.180000,44.106499,23.269114,17.170198,31.878227,27.201120,38.744705,15.196663,49.154808
2012-05-24,119.766667,25.419931,16.724091,12.789720,0.277647,39.303528,10.038571,36.349472,17.998116,43.402740,...,23.908106,31.540001,44.416229,23.237143,17.012516,32.904305,27.338070,38.999237,15.052042,49.496342


In [9]:
# Рассчитываем корреляции с статистической значимостью
from scipy.stats import pearsonr
ticker_columns = [col for col in combined.columns if col != 'fluxadjflux']

corr_list = []
for ticker in ticker_columns:
    r, p_value = pearsonr(combined['fluxadjflux'], combined[ticker])
    corr_list.append({
        'Ticker': ticker,
        'Correlation': r,
        'p-value': p_value,
        'Significant (5%)': p_value < 0.05
    })

corr_df = pd.DataFrame(corr_list).sort_values('Correlation', key=abs, ascending=False)

print("Детализированные результаты корреляции:")
print(corr_df)

Детализированные результаты корреляции:
   Ticker  Correlation        p-value  Significant (5%)
23    XOM     0.703125   0.000000e+00              True
3    NVDA     0.595199  1.881380e-306              True
12     GE     0.480626  6.820823e-185              True
7    INTC    -0.466321  9.739890e-173              True
18   AAPL     0.450733  3.889767e-160              True
1     WMT     0.438294  1.531897e-150              True
17   MSFT     0.415934  2.851840e-134              True
22  GOOGL     0.371646  1.691138e-105              True
0      KO     0.368140  2.094490e-103              True
4     DIS    -0.363325  1.425650e-100              True
8      PG     0.360829   4.010086e-99              True
10   TSLA     0.356313   1.559911e-96              True
14    JPM     0.341031   4.446032e-88              True
19   META     0.308040   2.183026e-71              True
11    PEP     0.302414   9.564231e-69              True
21     MA     0.288541   1.764255e-62              True
20      